In [1]:
#importing packages & libraries
import pandas as pd
from pandas import *
import requests
from bs4 import BeautifulSoup
import time

# iPortal Data

In [2]:
# EXCEL IMPORT & EXPORT 

# imports data from excel based on location, drops unneeded columns
file_path = 'C:/Users/matth/Desktop/Coding/WebScrape/'
iPortal_import = pd.read_excel(file_path + 'iPortal.xlsx', sheet_name = 'Assets')
iPortal_import = iPortal_import.drop(columns = ['Type', 'PLC', 'Rack', 'SubAddress', 'Slot', 'SubSlot', 'Module', 'Order number',
                                           'GSDML files', 'Online-MLFB', 'Online-HW version', 'Online serial No.', 'IP',
                                           'Subnet', 'Gateway', 'MAC', 'Device ID', 'Robot', 'Rob. Cell name', 'Rob. Controller', 
                                            'Rob. Serial.', 'Rob. KSS', 'ANZ. Ext. Axes', 'PN-stack active', 'ANZ. secure IOs', 'Number of IOs',
                                           'Version of PROFINET', 'MasterBusCycleTime', 'MasterBusTimeout', 'SlaveBusCycleTime', 
                                           'SlaveBusTimeout', 'Profienergy enabled', 'ProfiEnergyTimeToPauseHibernate',
                                           'ProfiEnergyTimeMinLengthOfStayHibernate', 'ProfiEnergyTimeToOperateHibernate',
                                           'ProfiEnergyTimeToPauseDrivesOff', 'ProfiEnergyTimeMinLengthOfStayDrivesOff',
                                           'ProfiEnergyTimeToOperateDrivesOff', 'PN product name', 'Active participants',
                                            'User-ID','Cycle time', 'SubSystem', 'Version', 'comment', 'Manufacturer'])
print(iPortal_import)

# filling in empty/NaN cells w/ value preceding the cell 
iPortal_data = iPortal_import.fillna(method='ffill')
print(iPortal_data)

# list of the names of all products
productList = iPortal_data['Product name'].unique()
pList = pd.DataFrame(productList)
print(pList)

# number of unique products 
len(productList)

# data frame that house the count of each device/product 
productCount = iPortal_data['Product name'].value_counts()
print(productCount)

# # export data to new excel
# writer = pd.ExcelWriter(file_path + 'allData.xlsx')
# iPortal_data.to_excel(writer, 'IPortal')
# pList.to_excel(writer, 'List of Products')
# productCount.to_excel(writer, 'Count per Product')
# writer.save()

             Product name Online FW version  Vendor ID  \
0                PNPN_IOC               NaN       42.0   
1                     NaN               NaN        NaN   
2                     NaN               NaN        NaN   
3                     NaN               NaN        NaN   
4                  S7-300          V 3.2.10       42.0   
5                     NaN               NaN        NaN   
6                     NaN               NaN        NaN   
7          SCALANCE X-400         V 3.7.1.2       42.0   
8                     NaN               NaN        NaN   
9                     NaN               NaN        NaN   
10                    NaN               NaN        NaN   
11                    NaN               NaN        NaN   
12                    NaN               NaN        NaN   
13                    NaN               NaN        NaN   
14                    NaN               NaN        NaN   
15                    NaN               NaN        NaN   
16            

# Siemens Scrape

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import *

#request to get html from website
r = requests.get('https://cert-portal.siemens.com/productcert/rss/advisories.atom')
soup = BeautifulSoup(r.text)


vulns = soup.find_all('entry')
data = {'Vendor': ['0'], 'CVSS Score': ['0'], 'Vector': ['0'], 'Products': ['0'], 'Link': ['0']}
master_df = pd.DataFrame(data=data)


##function to get info from each vuln
def scrape(vulns):
    url = str(vulns).split('<id>', 1)[1]
    url = url.split('</id>')[0]
    url = url.replace('pdf', 'txt', 2)
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    html = str(soup).lower()
    
    df = {'Vendor': ['0'], 'CVSS Score': ['0'], 'Vector': ['0'], 'Products': ['0'], 'Link': ['0']}
    df = pd.DataFrame(data=df)

    #setting values
    
    #name
    name = 'siemens'
    
    #link 
    link = url
    
    #cvss score
    if('base score:') in html:
        cvss = html.split('base score:', 1)[1]
        cvss = cvss.split('summary')[0]
        cvss = cvss.strip()
    else:
        cvss = 'NULL'
    
    #vector
    if('cvss vector') in html:
        vector = html.split('cvss vector', 1)[1]
        vector = vector.split('/', 1)[1]
        vector = vector.split('cwe')[0]
        vector = vector.strip()
    else:
        vector = 'NULL'
    
    #products
    if('affected products and solution') in html:
        products = html.split('affected products and solution', 1)[1]
        products = products.split('==============================', 1)[1]
        products = products.split('workarounds')[0]
        products = products.strip()
    else:
        products = 'NULL'
    
    data = {'Vendor': [name], 'CVSS Score': [cvss], 'Vector': [vector], 'Products': [products], 'Link': [link]}
    temp_df = pd.DataFrame(data=data)
    df = df.append(temp_df)
    df = df[df.Vendor != '0']
    
    return(df)

for i in range(len(vulns)):
    temp_df = scrape(vulns[i])
    master_df = master_df.append(temp_df)
    

siemens_df = master_df    
# #exporting df to excel to get a better look at the data
# master_df = master_df[master_df.Vendor != '0']
# file_path = 'C:/Users/matth/Desktop/Coding/WebScrape/'
# writer = pd.ExcelWriter(file_path + 'allData.xlsx')
# master_df.to_excel(writer, 'siemensScrape')
# writer.save()


https://cert-portal.siemens.com/productcert/txt/ssa-780073.txt
https://cert-portal.siemens.com/productcert/txt/ssa-232418.txt
https://cert-portal.siemens.com/productcert/txt/ssa-844761.txt
https://cert-portal.siemens.com/productcert/txt/ssa-508982.txt
https://cert-portal.siemens.com/productcert/txt/ssa-938930.txt
https://cert-portal.siemens.com/productcert/txt/ssa-270778.txt
https://cert-portal.siemens.com/productcert/txt/ssa-398519.txt
https://cert-portal.siemens.com/productcert/txt/ssa-431678.txt
https://cert-portal.siemens.com/productcert/txt/ssa-451445.txt
https://cert-portal.siemens.com/productcert/txt/ssa-750824.txt
https://cert-portal.siemens.com/productcert/txt/ssa-780073.txt
https://cert-portal.siemens.com/productcert/txt/ssa-232418.txt
https://cert-portal.siemens.com/productcert/txt/ssa-273799.txt
https://cert-portal.siemens.com/productcert/txt/ssa-349422.txt
https://cert-portal.siemens.com/productcert/txt/ssa-473245.txt
https://cert-portal.siemens.com/productcert/txt/ssa-462

https://cert-portal.siemens.com/productcert/txt/ssa-603476.txt
https://cert-portal.siemens.com/productcert/txt/ssa-899560.txt
https://cert-portal.siemens.com/productcert/txt/ssa-530931.txt
https://cert-portal.siemens.com/productcert/txt/ssb-439005.txt
https://cert-portal.siemens.com/productcert/txt/ssa-686531.txt
https://cert-portal.siemens.com/productcert/txt/ssa-473245.txt
https://cert-portal.siemens.com/productcert/txt/ssa-616472.txt
https://cert-portal.siemens.com/productcert/txt/ssa-462066.txt
https://cert-portal.siemens.com/productcert/txt/ssa-898181.txt
https://cert-portal.siemens.com/productcert/txt/ssa-434032.txt
https://cert-portal.siemens.com/productcert/txt/ssa-121293.txt
https://cert-portal.siemens.com/productcert/txt/ssa-275839.txt
https://cert-portal.siemens.com/productcert/txt/ssa-293562.txt
https://cert-portal.siemens.com/productcert/txt/ssa-346262.txt
https://cert-portal.siemens.com/productcert/txt/ssa-349422.txt
https://cert-portal.siemens.com/productcert/txt/ssa-462

https://cert-portal.siemens.com/productcert/txt/ssa-293562.txt
https://cert-portal.siemens.com/productcert/txt/ssa-306710.txt
https://cert-portal.siemens.com/productcert/txt/ssa-346262.txt
https://cert-portal.siemens.com/productcert/txt/ssa-348629.txt
https://cert-portal.siemens.com/productcert/txt/ssa-535640.txt
https://cert-portal.siemens.com/productcert/txt/ssa-559174.txt
https://cert-portal.siemens.com/productcert/txt/ssa-592007.txt
https://cert-portal.siemens.com/productcert/txt/ssb-439005.txt
https://cert-portal.siemens.com/productcert/txt/ssa-579309.txt
https://cert-portal.siemens.com/productcert/txt/ssa-325546.txt
https://cert-portal.siemens.com/productcert/txt/ssa-982399.txt
https://cert-portal.siemens.com/productcert/txt/ssa-181018.txt
https://cert-portal.siemens.com/productcert/txt/ssa-293562.txt
https://cert-portal.siemens.com/productcert/txt/ssa-348629.txt
https://cert-portal.siemens.com/productcert/txt/ssa-982399.txt
https://cert-portal.siemens.com/productcert/txt/ssa-170

https://cert-portal.siemens.com/productcert/txt/ssa-731239.txt
https://cert-portal.siemens.com/productcert/txt/ssa-901333.txt
https://cert-portal.siemens.com/productcert/txt/ssa-284673.txt
https://cert-portal.siemens.com/productcert/txt/ssa-701708.txt


# Sick AG Scrape

In [4]:
#request to get html from website
r = requests.get('https://www.sick.com/de/en/service-and-support/the-sick-product-security-incident-response-team-sick-psirt/w/psirt/#advisories & https://www.us-cert.gov/ics/advisories-by-vendor-last-revised-date?page=9')
soup = BeautifulSoup(r.text)


#gets the object that contians information on vulnerabilities
advisories = soup.find('tbody')

#essentailly breaks up all of the vulnerabilities into their rows and contains them in a list
items = advisories.find_all('tr')

#initialize dataframe
d = {'Vendor': ['0'], 'CVSS Score': ['0'],'Vector': ['0'], 'Products': ['0'], 'PDF': ['0']}
sick_vuln_df = pd.DataFrame(data=d)
sick_vuln_df

#for loop to add info from rows to dataframe, iterate through items
length = len(items)
index = 1
for i in range(length-1):
    
    #creating variables based on indexed information on the vulnerability
    work = items[index]
    tds = work.find_all('td')
    vendor = 'Sick AG'
    CVSS = tds[2].get_text()
    Products = tds[3].get_text()
    
    #PDF url needs to be converted into a string and made into a substring and then appended to base url
    PDF = str(tds[5])
    string = str(PDF)
    update = string.split('href="', 1)[1]
    final = update.split('"')[0]
    url = 'https://www.sick.com' + final
    
    #creating data set to create temp dataframe to append to master dataframe of vulnerabilities for SICK
    d = {'Vendor': [vendor], 'CVSS Score': [CVSS], 'Vector': 'NULL', 'Products': [Products], 'PDF': [url]}
    temp_df = pd.DataFrame(data=d)
    sick_vuln_df = sick_vuln_df.append(temp_df)
    index = index + 1
        
sick_vuln_df = sick_vuln_df.reset_index()
sick_vuln_df = sick_vuln_df.drop(columns = ['index'])
sick_vuln_df = sick_vuln_df.drop([0])
sick_vuln_df

# #exporting df to excel to get a better look at the data
# master_df = sick_vuln_df[sick_vuln_df.Vendor != '0']
# file_path = 'C:/Users/matth/Desktop/Coding/WebScrape/'
# writer = pd.ExcelWriter(file_path + 'allData.xlsx')
# master_df.to_excel(writer, 'sickAGScrape')
# writer.save()

,Vendor,CVSS Score,Vector,Products,PDF
1,Sick AG,9.8,NULL,MSC800 all versions,https://www.sick.com/medias/SCA-2019-0001.pdf?...
2,Sick AG,7.5,NULL,FX0-GPNT00000 (1044074)\nFX0-GENT00000 (104407...,https://www.sick.com/medias/SCA-2019-002.pdf?c...


# US-CERT Scrape

In [14]:
#request to get html from website
r = requests.get('https://www.us-cert.gov/ics/advisories-by-vendor-last-revised-date?page=0')
soup = BeautifulSoup(r.text)

#function that gets a pages html and then returns a list of all vendors html on the page
def vendorsPerPage(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    
    #getting a list of all vendors html on the page
    body = soup.find(role='main')
    div = body.find(lambda tag: tag.name == 'div' and tag['class'] == ['view-content'])
    item_list = div.find_all('div')
    return(item_list)



#function that pulls all links of a vendor
def vendorLinks(vendor_html):
    link_list = vendor_html.find_all('a', href=True)
    for i in range(len(link_list)):
        update = str(link_list[i]).split('href="', 1)[1]
        final = update.split('"')[0]
        link_list[i] = final
    return(link_list)



#function that scrapes data from link list and creates a df for the vendor
def vulnScrape(link_list, name):
    print(name)
    print(link_list)
    print(len(link_list))
    baseUrl = 'https://www.us-cert.gov'
    df = {'Vendor': ['0'], 'CVSS Score': ['0'], 'Vector': ['0'], 'Products': ['0'], 'Link': ['0']}
    df = pd.DataFrame(data=df)
    if len(link_list) >1:
        for i in range((len(link_list))):
            url = baseUrl + link_list[i]
            vuln_df = vulnInfo(url, name)
            df = df.append(vuln_df)
            
    else:
        print(link_list)
        link_list = str(link_list).split("['", 1)[1]
        link_list = link_list.split("']")[0]
        url = baseUrl + str(link_list)
        vuln_df = vulnInfo(url, name)
        df = df.append(vuln_df)

    df = df[df.Vendor != '0']        
    return(df)



#function that goes to vuln page and scrapes inf0
df = {'Vendor': ['0'], 'CVSS Score': ['0'], 'Vector': ['0'], 'Products': ['0'], 'Link': ['0']}
df = pd.DataFrame(data=df)

def vulnInfo(url, name):
    print(url)
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text)
    except:
        time.sleep(2)
        r = requests.get(url)
        soup = BeautifulSoup(r.text)
        
    vendor = name
    
    #parsing cvss score from html
    cvss = str(soup)
    cvss = cvss.lower()
    if('a cvss') in cvss:
        cvss = cvss.split('a cvss', 1)[1]
        if('score of') in cvss:
            cvss = cvss.split('score of ', 1)[1]
            cvss = cvss.split(' ')[0]
            cvss = cvss[0:3]
        else:
            cvss = cvss.split('score: ', 1)[1]
            cvss = cvss.split(' ')[0]
            cvss = cvss[0:3]
    else:
        cvss = 'NULL'
    
    #parsing vector from html
    vector = str(soup)
    vector = vector.lower()
    if('cvss vector') in vector:
        if('cvss vector string is') in vector:
            vector = vector.split('cvss vector string is', 1)[1]
            if ((vector.index('(')) == 0) and ((vector.index('a') == 1)):
                vector = vector.split('.')[0]
            elif ('(cvss:') in vector:
                vector = vector.split('/', 1)[1]
                vector = vector.split('.')[0]
                vector = '(' + vector
            elif('>a') in vector:
                vector = vector.split('/a', 1)[1]
                vector = vector.split('"')[0]
                vector = 'a' + vector
                if(')') in vector:
                    vector = vector.split(')')[0]
            else:
                vector = vector.split('>', 1)[1]
                vector = vector.split('<')[0]
        else:
            vector = vector.split('cvss vector string', 1)[1]
            vector = vector.split('is', 1)[1]
            if ((vector.index('(')) == 0) and ((vector.index('a') == 1)):
                vector = vector.split('.')[0]
            elif ('(cvss:') in vector:
                vector = vector.split('/', 1)[1]
                vector = vector.split('.')[0]
                vector = '(' + vector
            elif('>a') in vector:
                vector = vector.split('/a', 1)[1]
                vector = vector.split('"')[0]
                vector = 'a' + vector
                if(')') in vector:
                    vector = vector.split(')')[0]
            else:
                vector = vector.split('>', 1)[1]
                vector = vector.split('<')[0]
    else:
        vector = 'NULL'
    
    #parsing products affected from html
    products = str(soup)
    products = products.lower()
    if('affected products') in products:
        if('affected:') in products:
            print('1')
            products = products.split('affected:', 1)[1]
            products = products.split('</p>', 1)[1]
            if (products.index('u')) == 0:
                print('1a')
                products = products.split('<li>', 1)[1]
                products = products.split('</li>')[0]
            elif (products.index('p')) == 0:
                print('1b')
                print(products)
                if('<li>') in products:
                    products = products.split('<li>', 1)[1]
                    products = products.split('</li>')[0]
                elif('<p>') in products:
                    products = products.split('<p>', 1)[1]
                    products = products.split('</p')[0]
            else:
                if('<ul>') in products:
                    products = products.split('<ul>', 1)[1]
                    products = products.split('</ul>')[0]
                elif('<p>') in products:
                    products = products.split('<p>', 1)[1]
                    products = products.split('</p')[0]
        elif('affected products') in products:
            print('2')
            products = products.split('affected products', 1)[1]
            products = products.split('/h3', 1)[1]
            products = products.split('<h3>')[0]
        else:
            print('3')
            products = products.split('<p>', 1)[1]
            products = products.split('</p>')[0]
    else:
        products = 'NULL'
   
    #appending variables to df
    vuln = {'Vendor': [name], 'CVSS Score': [cvss], 'Vector': [vector], 'Products': [products], 'Link': [url]}
    vuln_df = pd.DataFrame(data=vuln)
    
    #dropping row in df with 0 values
    vuln_df = vuln_df.reset_index(drop=True)
    return(vuln_df)


#iterating through all pages of the website

#getting down to the code that holds the url for the last page of results
basePgUrl = 'https://www.us-cert.gov/ics/advisories-by-vendor-last-revised-date?page='
pg = soup.find_all(role = 'navigation')
buttons = pg[1].find_all('li')
string = str(buttons[10])
update = string.split('href="?page=', 1)[1]
final = update.split('"')[0]
numbOfPgs = int(final)
df = {'Vendor': ['0'], 'CVSS Score': ['0'], 'Vector': ['0'], 'Products': ['0'], 'Link': ['0']}
master_df = pd.DataFrame(data=df)

for i in range(numbOfPgs+1):
    pgUrl = basePgUrl + str(i) 
    item_list = vendorsPerPage(pgUrl)
    length = len(item_list)
    for x in range(length):
        print(x)
        vName = item_list[x].find('h3')
        name = vName.get_text()
        name = name.lower() 
        link_list = vendorLinks(item_list[x])
        df = vulnScrape(link_list, name)
        master_df = master_df.append(df)
        

# #exporting df to excel to get a better look at the data
# file_path = 'C:/Users/matth/Desktop/Coding/WebScrape/'
# writer = pd.ExcelWriter(file_path + 'allData.xlsx')
# master_df.to_excel(writer, 'US_CertScrape')
# writer.save()

us_cert_df = master_df
us_cert_df

0
360 systems
['/ics/advisories/ICSA-13-038-01A']
1
['/ics/advisories/ICSA-13-038-01A']
https://www.us-cert.gov/ics/advisories/ICSA-13-038-01A
1
1
bmc medical, 3b medical
['/ics/advisories/ICSMA-17-227-01']
1
['/ics/advisories/ICSMA-17-227-01']
https://www.us-cert.gov/ics/advisories/ICSMA-17-227-01
1
2
3s-smart software solutions
['/ics/advisories/icsa-19-255-01', '/ics/advisories/icsa-19-255-02', '/ics/advisories/icsa-19-255-03', '/ics/advisories/icsa-19-255-04', '/ics/advisories/icsa-19-255-05', '/ics/advisories/icsa-19-213-03', '/ics/advisories/icsa-19-213-04', '/ics/advisories/ICSA-18-352-03', '/ics/advisories/ICSA-18-352-04', '/ics/advisories/ICSA-14-030-01', '/ics/advisories/ICSA-12-006-01', '/ics/advisories/ICSA-13-011-01', '/ics/advisories/ICSA-13-050-01A', '/ics/advisories/ICSA-15-258-02', '/ics/advisories/ICSA-15-288-01', '/ics/advisories/ICSA-15-293-03', '/ics/advisories/ICSA-13-142-01', '/ics/advisories/ICSA-18-032-02', '/ics/advisories/ICSA-17-087-02']
19
https://www.us-ce

2
https://www.us-cert.gov/ics/advisories/ICSA-14-079-03
1
https://www.us-cert.gov/ics/advisories/ICSA-14-198-02
1
https://www.us-cert.gov/ics/advisories/ICSA-14-261-01
1
https://www.us-cert.gov/ics/advisories/ICSA-14-324-01
1
https://www.us-cert.gov/ics/advisories/ICSA-15-041-01
2
https://www.us-cert.gov/ics/advisories/ICSA-12-047-01A
2
https://www.us-cert.gov/ics/advisories/ICSA-15-251-01A
1
https://www.us-cert.gov/ics/advisories/ICSA-15-258-04
1
https://www.us-cert.gov/ics/advisories/ICSA-15-309-01
2
https://www.us-cert.gov/ics/advisories/ICSA-15-344-01
2
https://www.us-cert.gov/ics/advisories/ICSA-16-014-01
2
https://www.us-cert.gov/ics/advisories/ICSA-16-173-01
2
https://www.us-cert.gov/ics/advisories/ICSA-18-135-01
1
https://www.us-cert.gov/ics/advisories/ICSA-18-114-03
1
https://www.us-cert.gov/ics/advisories/ICSA-18-023-01
1
https://www.us-cert.gov/ics/advisories/ICSA-18-004-02A
2
https://www.us-cert.gov/ics/advisories/ICSA-17-306-02
1
https://www.us-cert.gov/ics/advisories/ICSA

1
30
binom3
['/ics/advisories/ICSA-17-031-01A']
1
['/ics/advisories/ICSA-17-031-01A']
https://www.us-cert.gov/ics/advisories/ICSA-17-031-01A
1
31
biosense webster
['/ics/advisories/ICSMA-18-107-02']
1
['/ics/advisories/ICSMA-18-107-02']
https://www.us-cert.gov/ics/advisories/ICSMA-18-107-02
1
32
black box
['/ics/advisories/ICSA-16-147-03']
1
['/ics/advisories/ICSA-16-147-03']
https://www.us-cert.gov/ics/advisories/ICSA-16-147-03
1
33
blf-tech llc
['/ics/advisories/ICSA-17-115-01']
1
['/ics/advisories/ICSA-17-115-01']
https://www.us-cert.gov/ics/advisories/ICSA-17-115-01
1
34
bmc medical, 3b medical
['/ics/advisories/ICSMA-17-227-01']
1
['/ics/advisories/ICSMA-17-227-01']
https://www.us-cert.gov/ics/advisories/ICSMA-17-227-01
1
35
boston scientific
['/ics/advisories/ICSMA-17-292-01']
1
['/ics/advisories/ICSMA-17-292-01']
https://www.us-cert.gov/ics/advisories/ICSMA-17-292-01
1
36
c3-ilex
['/ics/advisories/ICSA-12-271-01']
1
['/ics/advisories/ICSA-12-271-01']
https://www.us-cert.gov/ics/

1
https://www.us-cert.gov/ics/advisories/ICSA-18-172-01
1
https://www.us-cert.gov/ics/advisories/ICSA-18-137-04
1
https://www.us-cert.gov/ics/advisories/ICSA-18-151-01
1
https://www.us-cert.gov/ics/advisories/ICSA-18-116-01
1
https://www.us-cert.gov/ics/advisories/ICSA-18-060-03
1
https://www.us-cert.gov/ics/advisories/ICSA-18-058-02
1
https://www.us-cert.gov/ics/advisories/ICSA-18-004-01
1
https://www.us-cert.gov/ics/advisories/ICSA-16-348-03
1
17
detcon
['/ics/advisories/ICSA-17-136-01']
1
['/ics/advisories/ICSA-17-136-01']
https://www.us-cert.gov/ics/advisories/ICSA-17-136-01
1
18
digi international
['/ics/advisories/icsa-20-042-13', '/ics/advisories/ICSA-14-128-01']
2
https://www.us-cert.gov/ics/advisories/icsa-20-042-13
1
https://www.us-cert.gov/ics/advisories/ICSA-14-128-01
1
19
digital canal structural
['/ics/advisories/ICSA-17-157-02']
1
['/ics/advisories/ICSA-17-157-02']
https://www.us-cert.gov/ics/advisories/ICSA-17-157-02
1
20
digital electronics
['/ics/advisories/ICSA-12-17

1
https://www.us-cert.gov/ics/advisories/ICSA-17-073-01
2
https://www.us-cert.gov/ics/advisories/ICSA-16-350-01
1
5
festo
['/ics/advisories/ICSA-14-084-01']
1
['/ics/advisories/ICSA-14-084-01']
https://www.us-cert.gov/ics/advisories/ICSA-14-084-01
1
6
harman-kardon, fiat-chrysler automobile us llc
['/ics/advisories/ICSA-15-260-01', '/ics/advisories/ICSA-15-260-01']
2
https://www.us-cert.gov/ics/advisories/ICSA-15-260-01
1
https://www.us-cert.gov/ics/advisories/ICSA-15-260-01
1
7
fidelix
['/ics/advisories/ICSA-16-357-01']
1
['/ics/advisories/ICSA-16-357-01']
https://www.us-cert.gov/ics/advisories/ICSA-16-357-01
2
8
flexera
['/ics/advisories/icsa-19-323-01']
1
['/ics/advisories/icsa-19-323-01']
https://www.us-cert.gov/ics/advisories/icsa-19-323-01
1
9
fox-it
['/ics/advisories/ICSA-14-269-02']
1
['/ics/advisories/ICSA-14-269-02']
https://www.us-cert.gov/ics/advisories/ICSA-14-269-02
1
10
fr. sauter ag
['/ics/advisories/ICSA-18-305-04']
1
['/ics/advisories/ICSA-18-305-04']
https://www.us-c

...

2
https://www.us-cert.gov/ics/advisories/icsa-20-014-02
1
https://www.us-cert.gov/ics/advisories/icsa-20-014-03
1
https://www.us-cert.gov/ics/advisories/icsa-20-014-04
1
https://www.us-cert.gov/ics/advisories/icsa-20-014-05
1
https://www.us-cert.gov/ics/advisories/ICSA-18-165-01
2
https://www.us-cert.gov/ics/advisories/ICSA-18-163-02
2
https://www.us-cert.gov/ics/advisories/ICSA-19-162-04
2
https://www.us-cert.gov/ics/advisories/ICSA-19-085-01
1
https://www.us-cert.gov/ics/advisories/icsa-19-344-07
1
https://www.us-cert.gov/ics/advisories/icsa-19-281-03
1
https://www.us-cert.gov/ics/advisories/ICSA-12-349-01
1
https://www.us-cert.gov/ics/advisories/icsa-19-227-04
2
https://www.us-cert.gov/ics/advisories/icsa-19-344-01
1
https://www.us-cert.gov/ics/advisories/icsa-19-344-02
1
https://www.us-cert.gov/ics/advisories/icsa-19-344-03
1
https://www.us-cert.gov/ics/advisories/icsa-19-344-05
2
https://www.us-cert.gov/ics/advisories/icsa-19-318-02
1
https://www.us-cert.gov/ics/advisories/icsa-19

1
https://www.us-cert.gov/ics/advisories/ICSA-11-091-01A
2
https://www.us-cert.gov/ics/advisories/ICSA-14-105-03B
1
https://www.us-cert.gov/ics/advisories/ICSA-14-051-03B
1
https://www.us-cert.gov/ics/advisories/ICSA-14-087-01A
1
https://www.us-cert.gov/ics/advisories/ICSA-11-223-01A
2
https://www.us-cert.gov/ics/advisories/ICSA-14-205-02A
1
https://www.us-cert.gov/ics/advisories/ICSA-14-135-03A
1
https://www.us-cert.gov/ics/advisories/ICSA-13-338-01
2
https://www.us-cert.gov/ics/advisories/ICSA-15-013-01
1
https://www.us-cert.gov/ics/advisories/ICSA-15-020-01
1
https://www.us-cert.gov/ics/advisories/ICSA-15-022-01
1
https://www.us-cert.gov/ics/advisories/ICSA-15-034-01
2
https://www.us-cert.gov/ics/advisories/ICSA-15-034-02
1
https://www.us-cert.gov/ics/advisories/ICSA-14-329-02D
1
https://www.us-cert.gov/ics/advisories/ICSA-14-198-03G
1
https://www.us-cert.gov/ics/advisories/ICSA-15-048-02
1
https://www.us-cert.gov/ics/advisories/ICSA-15-048-01
1
https://www.us-cert.gov/ics/advisorie

2
https://www.us-cert.gov/ics/advisories/ICSA-11-356-01
2
1
sierra wireless
['/ics/advisories/ICSA-19-122-03', '/ics/advisories/ICSA-14-007-01B', '/ics/advisories/ICSA-16-105-01', '/ics/advisories/ICSA-17-115-02']
4
https://www.us-cert.gov/ics/advisories/ICSA-19-122-03
2
https://www.us-cert.gov/ics/advisories/ICSA-14-007-01B
1
https://www.us-cert.gov/ics/advisories/ICSA-16-105-01
1
https://www.us-cert.gov/ics/advisories/ICSA-17-115-02
1
2
ge, silex technology
['/ics/advisories/ICSMA-18-128-01']
1
['/ics/advisories/ICSMA-18-128-01']
https://www.us-cert.gov/ics/advisories/ICSMA-18-128-01
2
3
simplight
['/ics/advisories/ICSA-17-222-01']
1
['/ics/advisories/ICSA-17-222-01']
https://www.us-cert.gov/ics/advisories/ICSA-17-222-01
1
4
sinapsi
['/ics/advisories/ICSA-12-325-01', '/ics/advisories/ICSA-15-160-02']
2
https://www.us-cert.gov/ics/advisories/ICSA-12-325-01
1
https://www.us-cert.gov/ics/advisories/ICSA-15-160-02
1
5
sixnet
['/ics/advisories/ICSA-13-231-01B', '/ics/advisories/ICSA-16-14

1
44
abb, phoenix contact, schneider electric, siemens, wago
['/ics/advisories/ICSA-19-106-03']
1
['/ics/advisories/ICSA-19-106-03']
https://www.us-cert.gov/ics/advisories/ICSA-19-106-03
2
0
wago
['/ics/advisories/ICSA-12-249-02', '/ics/advisories/ICSA-18-198-02', '/ics/advisories/ICSA-18-088-01', '/ics/advisories/ICSA-18-044-01', '/ics/advisories/ICSA-16-357-02']
5
https://www.us-cert.gov/ics/advisories/ICSA-12-249-02
1
https://www.us-cert.gov/ics/advisories/ICSA-18-198-02
1
https://www.us-cert.gov/ics/advisories/ICSA-18-088-01
1
https://www.us-cert.gov/ics/advisories/ICSA-18-044-01
1
https://www.us-cert.gov/ics/advisories/ICSA-16-357-02
2
1
wecon
['/ics/advisories/icsa-19-353-03', '/ics/advisories/ICSA-19-036-03', '/ics/advisories/ICSA-18-277-01', '/ics/advisories/ICSA-18-212-03', '/ics/advisories/ICSA-18-261-01', '/ics/advisories/ICSA-16-189-01', '/ics/advisories/ICSA-18-116-02', '/ics/advisories/ICSA-17-103-01', '/ics/advisories/ICSA-17-353-05', '/ics/advisories/ICSA-18-011-01', '/

,Vendor,CVSS Score,Vector,Products,Link
0,0,0,0,0,0
0,360 systems,10.,ac:l/au:n/c:c/i:c/a:c,"<li>image server 2000 (all models),</li><li>im...",https://www.us-cert.gov/ics/advisories/ICSA-13...
0,"bmc medical, 3b medical",4.6,ac:l/pr:l/ui:n/s:u/c:n/i:l/a:l,"<li>luna cpap machine, all devices released pr...",https://www.us-cert.gov/ics/advisories/ICSMA-1...
0,3s-smart software solutions,7.5,av:n/ac:l/pr:n/ui:n/s:u/c:h/i:n/a:n,<li>codesys control for beaglebone</li>\n<li>c...,https://www.us-cert.gov/ics/advisories/icsa-19...
0,3s-smart software solutions,8.6,av:l/ac:l/pr:n/ui:r/s:c/c:h/i:h/a:h,>\n<p>all 32 and 64 bit codesys development sy...,https://www.us-cert.gov/ics/advisories/icsa-19...
0,3s-smart software solutions,8.8,av:n/ac:l/pr:l/ui:n/s:u/c:h/i:h/a:h,<li>codesys control for beaglebone</li>\n<li>c...,https://www.us-cert.gov/ics/advisories/icsa-19...
0,3s-smart software solutions,6.5,av:n/ac:l/pr:l/ui:n/s:u/c:n/i:n/a:h,<li>codesys control for beaglebone</li>\n<li>c...,https://www.us-cert.gov/ics/advisories/icsa-19...
0,3s-smart software solutions,7.5,av:n/ac:l/pr:n/ui:n/s:u/c:n/i:n/a:h,<li>codesys control for beaglebone</li>\n<li>c...,https://www.us-cert.gov/ics/advisories/icsa-19...
0,3s-smart software solutions,9.0,av:n/ac:h/pr:n/ui:n/s:c/c:h/i:h/a:h,>\n<p>all variants of the following codesys v3...,https://www.us-cert.gov/ics/advisories/icsa-19...
0,3s-smart software solutions,8.8,av:a/ac:l/pr:n/ui:n/s:u/c:h/i:h/a:h,>\n<p>all variants of the following codesys v3...,https://www.us-cert.gov/ics/advisories/icsa-19...


In [15]:
#appending all dfs together

master_df = siemens_df
master_df = master_df.append(siemens_df)

# Exporting everything to excel

In [17]:
# export data to new excel
writer = pd.ExcelWriter(file_path + 'allData.xlsx')
iPortal_data.to_excel(writer, 'IPortal')
pList.to_excel(writer, 'List of Products')
productCount.to_excel(writer, 'Count per Product')

siemens_df = siemens_df[siemens_df.Vendor != '0']
siemens_df.to_excel(writer, 'siemensScrape')

sick_vuln_df = sick_vuln_df[sick_vuln_df.Vendor != '0']
sick_vuln_df.to_excel(writer, 'sickAGScrape')

us_cert_df.to_excel(writer, 'US_CertScrape')

master_df.to_excel(writer, 'MasterFile')
writer.save()